In [1]:
import gc
import os
from os.path import exists
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('../input/lish-moa/train_features.csv')
test = pd.read_csv('../input/lish-moa/test_features.csv')
train['part']='train'
test['part']='test'
# df = pd.concat([tr, te], axis=0)
cs_c = ["#c0c", "#0cc", "#cc0", "#777"]
# del tr, te
gc.collect()

0

In [3]:
i_feats = ['sig_id']
x_feats = ['cp_type', 'cp_time', 'cp_dose']
g_feats = [f'g-{i}' for i in range(770)] # 772
c_feats = [f'c-{i}' for i in range(100)]
histbins = np.arange(-10, 10.5, .5)

In [3]:
cp_time = [24,48,72]
cp_dose = ['D1', 'D2']
cp_type = ["trt_cp", "ctl_vehicle"]

## cp_timeごとに、各C特徴で訓練/テストの分布を比較

In [4]:
def train_test_plot(tr, te, colname, colset, feats, suffix=""):
    res = len(feats) % 10
    if res != 0:
        feats = feats + feats[:(10-res)]
    feats_ = np.array(feats).reshape((-1,10))
    
    for batch_idx, feats_batch in enumerate(feats_):
        num_c = len(feats_batch)
        num_s = len(colset)
        fig, axes = plt.subplots(num_c, num_s, figsize=(5,10), sharex=True, sharey=True)
        fig.suptitle("train(magenta) vs test(yellow)")
        for j, ci in enumerate(feats_batch):
            print(f'batch {batch_idx:>3} / {len(feats_)}, plotting {j:>3} / {num_c:>3}', end='\r')
            for i, t in enumerate(colset):
                tr[tr[colname] == t][ci].hist(ax=axes[j][i], label="train", color=cs_c[0], histtype='step', linewidth=1, bins=histbins, density=True)
                te[te[colname] == t][ci].hist(ax=axes[j][i], label="test", color=cs_c[2], histtype='step', linewidth=1, bins=histbins, density=True)
                axes[j][i].grid(True)
            axes[j][0].set_ylabel(ci)
        for i, t in enumerate(colset):
            axes[0][i].set_title(f'time: {t}')
            axes[0][i].set_xlim(-10,10)
            
        dir_ = f"image/train-test/by_{colname}_{suffix}"
        if not exists(dir_):
            os.makedirs(dir_)
        plt.savefig(f"{dir_}/{feats_batch[0]}.pdf")
        plt.close()
    print()

## cp_timeごとのc特徴

In [5]:
train_test_plot(train, test, 'cp_time', cp_time, c_feats)

## cp_doseごとのc特徴

In [6]:
train_test_plot(train, test, 'cp_dose', cp_dose, c_feats)

## cp_typeごとのc特徴

In [7]:
train_test_plot(train, test, 'cp_type', cp_type, c_feats)

## trt_cpのみでcp_timeごとのc特徴、cp_doseごとのc特徴

In [8]:
train_test_plot(train[train['cp_type']=='trt_cp'], 
                test[test['cp_type']=='trt_cp'], 
                'cp_time', cp_time, c_feats, suffix="trt_cp")

In [9]:
train_test_plot(train[train['cp_type']=='trt_cp'], 
                test[test['cp_type']=='trt_cp'], 
                'cp_dose', cp_dose, c_feats, suffix="trt_cp")

## cp_dose条件下で、cp_timeごとのc特徴

In [10]:
train_test_plot(train[train['cp_dose']=='D1'], 
                test[test['cp_dose']=='D1'], 
                'cp_time', cp_time, c_feats, suffix="D1")
train_test_plot(train[train['cp_dose']=='D2'], 
                test[test['cp_dose']=='D2'], 
                'cp_time', cp_time, c_feats, suffix="D2")

## cp_time条件下で、cp_doseごとのc特徴

In [ ]:
train_test_plot(train[train['cp_time']==24],
               test[test['cp_time']==24],
               'cp_dose', cp_dose, c_feats, suffix="24")
train_test_plot(train[train['cp_time']==48],
               test[test['cp_time']==48],
               'cp_dose', cp_dose, c_feats, suffix="48")
train_test_plot(train[train['cp_time']==72],
               test[test['cp_time']==72],
               'cp_dose', cp_dose, c_feats, suffix="72")

---

## 統計量をプロット

---

## PCAによる特徴の再現性検証

In [11]:
# from sklearn.decomposition import PCA
# for i in range(3,30):
#     pca = PCA(n_components=i)
#     g_pca = pca.fit_transform(train[c_feats])
#     coverage = pca.explained_variance_ratio_.sum()
#     print(f"{i:>3}:{coverage:<.3}")
#     if coverage >= .9:
#         break